In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,regexp_replace,lit
import os

from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
spark = (
    SparkSession
    .builder
    .config("spark.jars", "./driver/postgresql-42.7.4.jar")
    .config("spark.drivers.extraClassPath","/usr/local/postgresql-42.7.4.jar")
    .config("spark.executor.extraClassPath","/usr/local/postgresql-42.7.4.jar")
    .master('local[*]')
    .appName('atividade-4')
    .getOrCreate()
)


your 131072x1 screen size is bogus. expect trouble
24/08/24 23:40:18 WARN Utils: Your hostname, BR-CND12708WZ resolves to a loopback address: 127.0.1.1; using 172.23.118.25 instead (on interface eth0)
24/08/24 23:40:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/08/24 23:40:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
path_raw = './Data/Raw'
path_trusted = './Data/Trusted'

df_bancos = spark.read.csv(f'{path_raw}/bancos.csv', header=True)
df_empregados1 = spark.read.csv(f'{path_raw}/empregados1.csv', header=True)
df_empregados2 = spark.read.csv(f'{path_raw}/empregados2.csv', header=True)

In [5]:
df_bancos = df_bancos.withColumn(
    "CNPJ",
    col("CNPJ")
    .cast("string")
    )
df_bancos = df_bancos.withColumn(
    'Nome',
    regexp_replace(
        col('Nome'),
        ' - PRUDENCIAL',
        '')
    )



In [7]:
df_bancos = df_bancos.withColumnsRenamed({
    "Segmento":"segmento",
    "CNPJ":"cnpj",
    "Nome":"nome"
})

In [8]:
df_bancos.write.parquet(f'{path_trusted}/bancos.parquet', mode='overwrite')

In [9]:
df_empregados1 = df_empregados1.withColumn(
    "CNPJ",
    col("CNPJ")
    .cast("string")
    )

In [10]:
df_empregados1 = df_empregados1.withColumn('Segmento', lit(""))
df_empregados2 = df_empregados2.withColumn('CNPJ',lit(""))

In [11]:
columns_empregados1 = df_empregados1.columns
df_empregados2 = df_empregados2.select(columns_empregados1)

df_empregados = df_empregados1.union(df_empregados2)
df_empregados = df_empregados.dropDuplicates()

In [17]:
df_empregados = df_empregados.withColumnsRenamed({
    "employer-website":"employer_website",
    "employer-headquarters":"employer_headquarters",
    "employer-founded":"employer_founded",
    "employer-industry":"employer_industry",
    "employer-revenue":"employer_revenue",
    "Geral":"geral",
    "Cultura e valores":"cultura_e_valores",
    "Diversidade e inclusão":"diversidade_e_inclusao",
    "Qualidade de vida":"qualidade_de_vida",
    "Alta liderança":"alta_lideranca",
    "Remuneração e benefícios":"remuneracao_e_beneficios",
    "Oportunidades de carreira":"oportunidades_de_carreira",
    "Recomendam para outras pessoas(%)":"recomendam_para_outras_pessoas",
    "Perspectiva positiva da empresa(%)":"perspectiva_positiva_da_empresa",
    "CNPJ":"cnpj",
    "Nome":"nome",
    "Segmento":"segmento"
})

In [19]:
df_empregados.write.parquet(f'{path_trusted}/empregados.parquet', mode='overwrite')


In [20]:
all_reclamacoes = os.listdir(path_raw)
all_reclamacoes = [file for file in all_reclamacoes if 'reclamacoes' in file]

In [21]:
lista_dataframe = []
for filename in all_reclamacoes:
    try:
        # Load the CSV file into a DataFrame
        df = spark.read.csv(f'{path_raw}/{filename}', header=True)
        
    except Exception as e:
        print(f'Não foi possível abrir o arquivo {filename}: {str(e)}')
        pass
    lista_dataframe.append(df)



In [22]:
df_reclamacoes = lista_dataframe[0]
for df in lista_dataframe[1:]:
    df_reclamacoes = df_reclamacoes.union(df)

In [23]:
df_reclamacoes = df_reclamacoes.drop('_c14')

In [25]:
df_reclamacoes = df_reclamacoes.withColumnsRenamed({
    "Ano":"ano",
    "Trimestre":"trimestre",
    "Tipo":"tipo",
    "CNPJ IF":"cnpj_if",
    "Instituição financeira":"instituicao_financeira",
    "Índice":"indice",
    "Quantidade de reclamações reguladas procedentes":"quantidade_de_reclamacoes_reguladas_procedentes",
    "Quantidade de reclamações reguladas - outras":"quantidade_de_reclamacoes_reguladas_outras",
    "Quantidade de reclamações não reguladas":"quantidade_de_reclamacoes_nao_reguladas",
    "Quantidade total de reclamações":"quantidade_total_de_reclamacoes",
    "Quantidade total de clientes \x96 CCS e SCR":"quantidade_total_de_clientes_ccs_e_scr",
    "Quantidade de clientes \x96 CCS":"quantidade_de_clientes_ccs",
    "Quantidade de clientes \x96 SCR":"quantidade_de_clientes_scr"
})

In [27]:
df_reclamacoes.write.parquet(f'{path_trusted}/reclamacoes.parquet', mode='overwrite')

In [28]:
url = f"jdbc:postgresql://localhost:{os.getenv('PORT')}/{os.getenv('DATABASE')}"
properties = {
    "user": os.getenv("USER_DB"),
    "password": os.getenv("PASS"),
    "driver": "org.postgresql.Driver"
}

In [29]:
df_bancos.write.jdbc(url=url, table="bancos", mode='overwrite', properties=properties)
df_empregados.write.jdbc(url=url, table="empregados", mode='overwrite', properties=properties)
df_reclamacoes.write.jdbc(url=url, table="reclamacoes", mode='overwrite', properties=properties)

24/08/24 23:59:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [30]:
properties

{'user': 'postgres', 'password': 'postgres', 'driver': 'org.postgresql.Driver'}

In [31]:
url

'jdbc:postgresql://localhost:5432/ingestao'

In [32]:
spark.stop()